# Data Gathering

In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

In [4]:
def handle_file(path:str, name:str)->str:
    '''
    routine to rename any conflicting file to prevent unwanted overriding.
    '''
    file = os.path.join(path, name)
    
    if os.path.isfile(file):
        print('Warning !')
        print('A file exists with the same name as the one being saved')
        keep = input('Should I override the existing file (y/n): ').strip().lower()
        
        if not (keep == 'y' or keep == 'yes'):

            new_name = name + '.backup'
            new_file = os.path.join(path, new_name)
            print(f'Keeping existing file as {new_file}')
            os.rename(file, new_file)
        
    return file

### [Residential Construction Spending](https://www.census.gov/construction/c30/c30index.html)

In [5]:
cons_spend = pd.read_excel('https://www.census.gov/construction/c30/xls/residentialsa.xls',
                          skiprows=[0,1])

## restrict data till 2021 only as 2022 has a lot of missing values

cons_spend = cons_spend.iloc[0:29, :]

## automatic conversion to suitable dtypes
cons_spend = cons_spend.convert_dtypes()

In [6]:
cons_spend.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,1993,194150,194689,190185,198296,198835,203175,208760,212238,214446,214778,227235,235648
1,1994,238612,236577,235150,244746,244056,244032,245463,243134,242619,234905,239857,241395
2,1995,241184,234257,228013,225488,222062,218972,224113,226045,228688,227589,231913,235572
3,1996,240958,243695,249952,252836,259296,263085,263549,263618,265677,261444,259668,253234
4,1997,255365,258595,264566,257490,264581,262299,261593,263951,270511,272980,272786,269192


__Parse to a proper format__

In [7]:
cons_spend = cons_spend.set_index('Year').stack().reset_index().rename(columns={'Year':'year', 'level_1':'month', 0:'spend'})
cons_spend.head()

,year,month,spend
0,1993,Jan,194150
1,1993,Feb,194689
2,1993,Mar,190185
3,1993,Apr,198296
4,1993,May,198835


In [8]:
## parse datetime

cons_spend['date'] = cons_spend['year'].apply(str) + '-' + cons_spend['month']

cons_spend['date'] = pd.to_datetime(cons_spend['date'])

cons_spend.drop(['year', 'month'], axis=1, inplace = True)

cons_spend.set_index(['date'], inplace=True)

cons_spend.head()

,spend
date,
1993-01-01,194150
1993-02-01,194689
1993-03-01,190185
1993-04-01,198296
1993-05-01,198835


### [Single Unit Building Permits](https://www.huduser.gov/Portal/ushmc/hs_sfm.html)

In [9]:
hud_supply_df = pd.read_excel('https://www.huduser.gov/portal/periodicals/ushmc/charts/HousingSupply.xlsx',
                        sheet_name='Construction_Monthly')

In [10]:
hud_supply_df

,Month,Auth_US,Auth_S1,Auth_S2to4,Auth_S5more,Auth_NE,Auth_MW,Auth_S,Auth_W,Starts_US,...,UndCon_MW,UndCon_S,UndCon_W,Complt_US,Complt_S1,Complt_S5more,Complt_NE,Complt_MW,Complt_S,Complt_W
0,1968-01-01,1179,679,81,419,181,333,414,251,1380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1968-02-01,1342,711,89,542,224,381,460,277,1520,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1968-03-01,1370,691,80,599,297,332,454,287,1466,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1968-04-01,1286,675,81,530,243,330,446,267,1554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1968-05-01,1297,665,83,549,218,369,435,275,1408,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,2022-01-01,1841,1197,57,587,151,270,958,462,1666,...,202.0,713.0,439.0,1247.0,929.0,305.0,84.0,139.0,670.0,354.0
649,2022-02-01,1857,1204,54,599,179,249,954,475,1777,...,206.0,728.0,447.0,1380.0,1076.0,296.0,133.0,196.0,759.0,292.0
650,2022-03-01,1879,1163,56,660,185,260,972,462,1716,...,216.0,739.0,462.0,1366.0,1052.0,303.0,108.0,192.0,793.0,273.0
651,2022-04-01,1823,1109,56,658,163,250,989,421,1810,...,217.0,761.0,469.0,1343.0,1015.0,313.0,97.0,170.0,737.0,339.0


In [11]:
## remove useless rows and columns

permits = hud_supply_df[['Month', 'Auth_S1']].copy()

permits.columns = ['date', 'permits']

In [12]:
## date column is already in datetime format

permits['date'] = pd.to_datetime(permits['date'])

permits['date'] = permits['date'].apply(lambda dt: dt.replace(day=1))

permits.set_index(['date'], inplace=True)

In [13]:
permits

,permits
date,
1968-01-01,679
1968-02-01,711
1968-03-01,691
1968-04-01,675
1968-05-01,665
...,...
2022-01-01,1197
2022-02-01,1204
2022-03-01,1163


### [New Housing Starts](https://www.huduser.gov/Portal/ushmc/hs_sfm.html)

In [14]:
new_starts = hud_supply_df[['Month', 'Starts_S1']].copy()

new_starts.columns = ['date', 'starts']

new_starts['date'] = new_starts['date'].apply(lambda dt: dt.replace(day=1))

new_starts.set_index(['date'], inplace=True)

In [15]:
new_starts

,starts
date,
1968-01-01,839
1968-02-01,993
1968-03-01,912
1968-04-01,925
1968-05-01,850
...,...
2022-01-01,1157
2022-02-01,1213
2022-03-01,1191


### [Housing completions](https://www.huduser.gov/Portal/ushmc/hs_sfm.html)

In [16]:
completions = hud_supply_df[['Month', 'Complt_S1']].copy()

completions.columns = ['date', 'completions']

completions['date'] = completions['date'].apply(lambda dt: dt.replace(day=1))

completions.set_index(['date'], inplace=True)


In [17]:
completions

,completions
date,
1968-01-01,NaN
1968-02-01,NaN
1968-03-01,NaN
1968-04-01,NaN
1968-05-01,NaN
...,...
2022-01-01,929.0
2022-02-01,1076.0
2022-03-01,1052.0


### [Manufactured Housing](https://www.huduser.gov/portal/ushmc/hs_man_hsg.html)

In [18]:
manufactured = pd.read_excel('https://www.huduser.gov/portal/periodicals/ushmc/charts/HousingSupply.xlsx',
                        sheet_name='Manufactured_Monthly')

In [19]:
manufactured = manufactured.loc[:, ['Month', 'Shipments_US']]

manufactured.columns = ['date', 'manufactured']

manufactured['date'] = manufactured['date'].apply(lambda dt: dt.replace(day=1))
manufactured.set_index(['date'], inplace=True)


In [20]:
manufactured

,manufactured
date,
1978-01-01,293
1978-02-01,275
1978-03-01,277
1978-04-01,267
1978-05-01,276
...,...
2022-01-01,116
2022-02-01,117
2022-03-01,119


### [New Housing for sale](https://www.huduser.gov/portal/ushmc/hs_newsf.html)

In [21]:
new_for_sale = pd.read_excel('https://www.huduser.gov/portal/periodicals/ushmc/charts/HousingSupply.xlsx',
                        sheet_name='NewForSale_Monthly')

/home/lambda/miniconda3/envs/zero/lib/python3.10/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [22]:
new_for_sale = new_for_sale.loc[:, ['Month', 'NewForSale_US', 'NewMnSply_US']]

new_for_sale.columns = ['date', 'new_for_sale', 'months_supply']


new_for_sale['date'] = new_for_sale['date'].apply(lambda dt: dt.replace(day=1))

new_for_sale.set_index(['date'], inplace=True)

In [23]:
new_for_sale

,new_for_sale,months_supply
date,,
1963-01-01,234,4.7
1963-02-01,236,6.6
1963-03-01,246,6.4
1963-04-01,254,5.3
1963-05-01,254,5.1
...,...,...
2022-01-01,394,5.7
2022-02-01,396,6.0
2022-03-01,411,6.9


### [Permit Value](https://www.census.gov/construction/bps/)

In [24]:
permit_value = pd.read_excel('https://www.census.gov/construction/bps/permitsbyusreg_cust.xls',
                             sheet_name='Val Monthly',
                            skiprows=list(range(4)), header=[0,1])

In [25]:
## drop unnecessary columns

permit_value = permit_value.iloc[:-6, [1, 3]]

permit_value.columns = ['date', 'permit_val']

In [26]:
## parse datetime

permit_value['date'] = pd.to_datetime(permit_value['date'], format='%Y-%m-%d')

permit_value['date'] = permit_value['date'].apply(lambda dt: dt.replace(day=1))

permit_value.set_index(['date'], inplace=True)

In [27]:
permit_value

,permit_val
date,
1988-01-01,4081.302
1988-02-01,5050.834
1988-03-01,7787.515
1988-04-01,7595.054
1988-05-01,8119.532
...,...
2022-02-01,23557.156
2022-03-01,28977.223
2022-04-01,27541.115


### [Vacancy Rate](https://www.census.gov/housing/hvs/data/histtabs.html) (Quarterly)

In [28]:
vacancy_df = pd.read_excel('https://www.census.gov/housing/hvs/files/qtr222/tab1.xlsx',
                       skiprows=range(4), header=[0,1,2])

In [29]:
vacancy_df.columns = vacancy_df.columns.droplevel(1)
vacancy_df = vacancy_df.iloc[3:-7]

arrays = [['rent_vacancy', 'owner_vacancy'], ['Q1', 'Q2', 'Q3', 'Q4']]

columns = pd.MultiIndex.from_product(arrays)
columns = columns.insert(0, ('year', ''))

vacancy_df.columns = columns


In [30]:
vacancy_df.dropna(axis=0, how='all', inplace=True)

mask = vacancy_df['year'].str.find('r') !=-1

repeat_cols = vacancy_df['year'][mask].str[:4].values

drop_index = vacancy_df[vacancy_df['year'].apply(lambda x: str(x)[:4] in repeat_cols and str(x).find('r') == -1)].index


vacancy_df = vacancy_df.drop(axis=0, index=drop_index).reset_index(drop=True)

vacancy_df['year'] = vacancy_df['year'].str[:4]

vacancy_df = vacancy_df.convert_dtypes()

vacancy_df.sort_values(['year'], ascending=True, inplace=True, ignore_index=True)


In [31]:
vacancy_df.head()

year rent_vacancy                owner_vacancy               
                  Q1   Q2   Q3   Q4            Q1   Q2   Q3   Q4
0  1965          8.5  8.2  7.8  8.5           1.7  1.5  1.6  1.5
1  1966          8.3  7.4  7.4  7.7           1.5  1.5  1.4  1.3
2  1967          7.3  6.9  7.0  6.2           1.4  1.3  1.4  1.3
3  1968          6.1  6.2  5.9  5.4           1.1  1.1  1.2  1.2
4  1969          5.6  5.7  5.5  5.1           1.0  1.0  1.1  1.0

In [32]:
vacancy_df = vacancy_df.set_index('year').stack().reset_index().rename(columns={'level_1': 'quarter'})

vacancy_df['date'] = vacancy_df['year'] + '-' + vacancy_df['quarter']

vacancy_df.drop(['year', 'quarter'], axis=1, inplace=True)

vacancy_df['date'] = pd.to_datetime(vacancy_df['date'])

vacancy_df.set_index(['date'], inplace=True)

In [33]:
vacancy_df.head()

,owner_vacancy,rent_vacancy
date,,
1965-01-01,1.7,8.5
1965-04-01,1.5,8.2
1965-07-01,1.6,7.8
1965-10-01,1.5,8.5
1966-01-01,1.5,8.3


In [34]:
### upsample to monthly date using nearest neighbor

vacancy_monthly = vacancy_df.resample('M').nearest()
vacancy_monthly.head()

,owner_vacancy,rent_vacancy
date,,
1965-01-31,1.7,8.5
1965-02-28,1.5,8.2
1965-03-31,1.5,8.2
1965-04-30,1.5,8.2
1965-05-31,1.6,7.8


### [Employment-population ratio](https://fred.stlouisfed.org/series/EMRATIO)

In [35]:
emratio = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=EMRATIO&scale=left&cosd=1948-01-01&coed=2022-07-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1948-01-01')

In [36]:
emratio.columns = ['date', 'emratio']


emratio['date'] = pd.to_datetime(emratio['date'], format='%Y-%m-%d')

emratio.set_index(['date'], inplace=True)

In [37]:
emratio

,emratio
date,
1948-01-01,56.6
1948-02-01,56.7
1948-03-01,56.1
1948-04-01,56.7
1948-05-01,56.2
...,...
2022-03-01,60.1
2022-04-01,60.0
2022-05-01,60.1


### [Population Level](https://fred.stlouisfed.org/series/CNP16OV)

In [38]:
pop_level = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=CNP16OV&scale=left&cosd=1948-01-01&coed=2022-07-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1948-01-01')

In [39]:
pop_level.columns = ['date', 'pop_level']

pop_level['date'] = pd.to_datetime(pop_level['date'], format='%Y-%m-%d')

pop_level.set_index(['date'], inplace=True)

In [40]:
pop_level

,pop_level
date,
1948-01-01,102603
1948-02-01,102698
1948-03-01,102771
1948-04-01,102831
1948-05-01,102923
...,...
2022-03-01,263444
2022-04-01,263559
2022-05-01,263679


### [Net Migration - Metropolitan Areas](https://www.census.gov/data/tables/time-series/demo/geographic-mobility/historic.html) (Annual)

In [41]:
migration_sheet = pd.read_excel('https://www2.census.gov/programs-surveys/demo/tables/geographic-mobility/time-series/historic/hst_mig_a_1.xlsx',
                            skiprows=[0,1,2,7],
                               header=[0,1,2,3])

In [42]:
migration_df = migration_sheet.iloc[0:-20, [0, 3]]

migration_df.columns = ['period', 'movers']

migration_df = migration_df.dropna(axis=0, subset=['period']).reset_index(drop=True)

idx = migration_df[migration_df['period'] == 'Percent'].index

migration_df = migration_df.loc[idx[0]+1:].reset_index(drop=True)

In [43]:
migration_df

,period,movers
0,2020-2021,8.4
1,2019-2020,9.3
2,2018-2019,9.8
3,2017-2018,10.1
4,2016-2017,11.0
...,...,...
66,1951-1952,20.3
67,1950-1951,21.2
68,1949-1950,19.1
69,1948-1949,19.2


In [44]:
migration_df[migration_df['period'].apply(lambda x: len(x) > 9)]

,period,movers
7,"2013-2014 (98,000 address file)1",11.5
10,2010-2011 (2010 controls)2,11.6
11,2010-2011 (2000 controls)3,11.6
12,2009-2010 (2010 controls)2,12.5
13,2009-2010 (2000 controls)3,12.5
18,2004-20054,13.9
19,2003-20045,13.7
22,2000-2001 (2000 controls)3,14.2
23,"2000-2001 (SCHIP, 2000 controls)6",14.5
24,2000-2001 (1990 controls)7,14.2


### [GDP](https://fred.stlouisfed.org/graph/?g=znfe)

In [45]:
gdp = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=off&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=USALORSGPNOSTSAM&scale=left&cosd=1990-01-01&coed=2022-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1960-01-01')

In [46]:
gdp.columns = ['date', 'gdp']

gdp['date'] = pd.to_datetime(gdp['date'], format='%Y-%m-%d')

gdp.set_index(['date'], inplace=True)

In [47]:
gdp

,gdp
date,
1990-01-01,101.321696
1990-02-01,101.342709
1990-03-01,101.336940
1990-04-01,101.292929
1990-05-01,101.202611
...,...
2022-01-01,100.193185
2022-02-01,100.069211
2022-03-01,99.916488


### [Mortgage Rate](https://fred.stlouisfed.org/graph/?g=zneW)

In [48]:
mortgage_rate = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=MORTGAGE30US&scale=left&cosd=1990-01-01&coed=2022-08-18&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1971-04-02')


In [49]:
mortgage_rate.columns = ['date', 'mortgage_rate']

mortgage_rate['date'] = pd.to_datetime(mortgage_rate['date'], format='%Y-%m-%d')

mortgage_rate.set_index(['date'], inplace=True)

In [50]:
mortgage_rate

,mortgage_rate
date,
1990-01-01,9.895000
1990-02-01,10.197500
1990-03-01,10.268000
1990-04-01,10.370000
1990-05-01,10.477500
...,...
2022-04-01,4.982500
2022-05-01,5.230000
2022-06-01,5.522000


### [Federal funds rate](https://fred.stlouisfed.org/series/DFF#0)

In [51]:
fed_fund_rate = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=DFF&scale=left&cosd=1954-07-01&coed=2022-08-19&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily%2C%207-Day&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1954-07-01')

In [52]:
fed_fund_rate.columns = ['date', 'fed_fund_rate']

fed_fund_rate['date'] = pd.to_datetime(fed_fund_rate['date'], format='%Y-%m-%d')

fed_fund_rate.set_index(['date'], inplace=True)

In [53]:
fed_fund_rate

,fed_fund_rate
date,
1954-07-01,1.13
1954-07-02,1.25
1954-07-03,1.25
1954-07-04,1.25
1954-07-05,0.88
...,...
2022-08-15,2.33
2022-08-16,2.33
2022-08-17,2.33


### [Delinquency Rate](https://fred.stlouisfed.org/series/DRSFRMACBS) (quaterly)

In [54]:
delinquent_rate = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=DRSFRMACBS&scale=left&cosd=1991-01-01&coed=2022-04-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly%2C%20End%20of%20Period&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1991-01-01')

In [55]:
delinquent_rate.columns = ['date', 'delinquent_rate']

delinquent_rate['date'] = pd.to_datetime(delinquent_rate['date'], format='%Y-%m-%d')

delinquent_rate.set_index(['date'], inplace=True)

In [56]:
delinquent_rate

,delinquent_rate
date,
1991-01-01,3.09
1991-04-01,3.17
1991-07-01,3.23
1991-10-01,3.29
1992-01-01,3.12
...,...
2021-04-01,2.48
2021-07-01,2.31
2021-10-01,2.28


### [Real Disposable Income](https://fred.stlouisfed.org/series/DSPIC96#0)

In [57]:
disp_income = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=DSPIC96&scale=left&cosd=1959-01-01&coed=2022-06-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1959-01-01')

In [58]:
disp_income.columns = ['date', 'disp_income']

disp_income['date'] = pd.to_datetime(disp_income['date'], format='%Y-%m-%d')

disp_income.set_index(['date'], inplace=True)

In [59]:
disp_income

,disp_income
date,
1959-01-01,2191.5
1959-02-01,2198.1
1959-03-01,2210.7
1959-04-01,2225.0
1959-05-01,2237.1
...,...
2022-02-01,15173.6
2022-03-01,15119.6
2022-04-01,15148.6


### [Median weekly nominal earnings](https://fred.stlouisfed.org/series/LEU0252887700Q#0) (quaterly)

In [60]:
week_earning = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=LEU0252887700Q&scale=left&cosd=2000-01-01&coed=2022-04-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=2000-01-01')

In [61]:
week_earning.columns = ['date', 'week_earning']

week_earning['date'] = pd.to_datetime(week_earning['date'], format='%Y-%m-%d')

week_earning.set_index(['date'], inplace=True)

In [62]:
week_earning

,week_earning
date,
2000-01-01,603
2000-04-01,606
2000-07-01,611
2000-10-01,614
2001-01-01,620
...,...
2021-04-01,1048
2021-07-01,1068
2021-10-01,1069


In [63]:
## linear interpolation to create monthly data


week_earning_monthly = week_earning.resample('MS').interpolate()
week_earning_monthly

,week_earning
date,
2000-01-01,603.000000
2000-02-01,604.000000
2000-03-01,605.000000
2000-04-01,606.000000
2000-05-01,607.666667
...,...
2021-12-01,1089.666667
2022-01-01,1100.000000
2022-02-01,1102.333333


### [Personal Savings](https://fred.stlouisfed.org/series/PMSAVE)

In [64]:
pm_save = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=PMSAVE&scale=left&cosd=1959-01-01&coed=2022-06-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1959-01-01')

In [65]:
pm_save.columns = ['date', 'pm_save']

pm_save['date'] = pd.to_datetime(pm_save['date'], format='%Y-%m-%d')

pm_save.set_index(['date'], inplace=True)

In [66]:
pm_save

,pm_save
date,
1959-01-01,39.6
1959-02-01,37.5
1959-03-01,36.5
1959-04-01,40.0
1959-05-01,38.1
...,...
2022-02-01,1060.0
2022-03-01,961.2
2022-04-01,950.1


### [Personal Consumption Expenditures: Durable Goods](https://fred.stlouisfed.org/series/PCEDG)

In [67]:
consump_durable = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=PCEDG&scale=left&cosd=1959-01-01&coed=2022-06-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-22&revision_date=2022-08-22&nd=1959-01-01')

In [68]:
consump_durable.columns = ['date', 'consump_durable']

consump_durable['date'] = pd.to_datetime(consump_durable['date'], format='%Y-%m-%d')

consump_durable.set_index(['date'], inplace=True)

In [69]:
consump_durable

,consump_durable
date,
1959-01-01,42.3
1959-02-01,44.2
1959-03-01,44.4
1959-04-01,45.1
1959-05-01,45.4
...,...
2022-02-01,2116.4
2022-03-01,2112.8
2022-04-01,2146.5


### [HomeOwnership Rate](https://www.huduser.gov/portal/ushmc/hi_HOR.html) (quarterly)

In [70]:
hor_qtr = pd.read_excel('https://www.huduser.gov/portal/periodicals/ushmc/charts/HousingOwnershipVacancy.xlsx',
                                   sheet_name='HOR_Quarterly')

In [71]:
hor_qtr = hor_qtr[['Quarter', 'HOR_US']]

hor_qtr.columns = ['date', 'hor']

In [72]:
### parse dates

hor_qtr['date'] = pd.to_datetime(hor_qtr['date'].apply(lambda x: '-'.join(x.strip().split())))

hor_qtr.set_index(['date'], inplace=True)

In [73]:
hor_qtr

,hor
date,
1997-01-01,65.4
1997-04-01,65.7
1997-07-01,66.0
1997-10-01,65.7
1998-01-01,65.9
...,...
2021-01-01,65.6
2021-04-01,65.4
2021-07-01,65.4


### [New Homes Sold](https://www.huduser.gov/portal/periodicals/ushmc/charts/HousingDemand.xlsx)

In [74]:
demand_hud = pd.read_excel('https://www.huduser.gov/portal/periodicals/ushmc/charts/HousingDemand.xlsx',
                          sheet_name='Sales_Monthly')

In [75]:
new_sold = demand_hud[['Month', 'NewSold_US']].copy()
new_sold.columns = ['date', 'new_sold']

new_sold['date'] = pd.to_datetime(new_sold['date'])

new_sold.set_index(['date'], inplace=True)

In [76]:
new_sold

,new_sold
date,
1963-01-01,591
1963-02-01,464
1963-03-01,461
1963-04-01,605
1963-05-01,586
...,...
2022-01-12,831
2022-02-13,790
2022-03-01,715


### []()

In [77]:
hp_idx = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1168&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=CSUSHPISA&scale=left&cosd=1987-01-01&coed=2022-05-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2022-08-23&revision_date=2022-08-23&nd=1987-01-01')


In [78]:
hp_idx.columns = ['date', 'hp_idx']

hp_idx['date'] = pd.to_datetime(hp_idx['date'])

hp_idx.set_index(['date'], inplace=True)

In [79]:
hp_idx

,hp_idx
date,
1987-01-01,63.965
1987-02-01,64.424
1987-03-01,64.735
1987-04-01,65.132
1987-05-01,65.565
...,...
2022-01-01,284.767
2022-02-01,290.371
2022-03-01,296.496


In [80]:
### downsampling to quarter period index

hp_idx_qtr = hp_idx.resample('QS').mean()

In [81]:
hp_idx_qtr

,hp_idx
date,
1987-01-01,64.374667
1987-04-01,65.590000
1987-07-01,66.926000
1987-10-01,68.117000
1988-01-01,69.253667
...,...
2021-04-01,254.633000
2021-07-01,266.878333
2021-10-01,276.591000


# Combining Data

### Monthly datasets

In [82]:
### monthly data frames
monthly_df = [
            permits,
            permit_value,
            new_starts,
            completions,
            manufactured,
            new_for_sale,
            emratio,
            pop_level,
            gdp,
            mortgage_rate,
            fed_fund_rate,
            disp_income,
            pm_save,
            consump_durable,
            new_sold,
            ]

In [83]:
monthly_1990 = cons_spend.join(monthly_df, how='inner')

In [84]:
monthly_1990 = monthly_1990.loc[:'2020-12-01']

In [85]:
monthly_1990

,spend,permits,permit_val,starts,completions,manufactured,new_for_sale,months_supply,emratio,pop_level,gdp,mortgage_rate,fed_fund_rate,disp_income,pm_save,consump_durable,new_sold
date,,,,,,,,,,,,,,,,,
1993-01-01,194150,989,5383.028,1091,975.0,248,265,5.4,61.4,193962,99.759963,7.9925,2.66,7237.6,425.6,538.1,596
1993-02-01,194689,953,5844.533,1063,1043.0,248,263,5.3,61.4,194108,99.679238,7.6825,3.25,7271.8,441.5,524.2,604
1993-03-01,190185,881,8227.729,950,983.0,241,268,5.4,61.5,194248,99.598357,7.4950,3.65,7249.2,439.0,521.2,602
1993-04-01,198296,922,8918.858,1110,1067.0,241,270,4.7,61.5,194398,99.525442,7.4720,3.31,7286.8,436.4,543.0,701
1993-05-01,198835,911,8320.702,1128,973.0,240,273,5.3,61.7,194549,99.466172,7.4650,3.01,7276.3,415.8,552.0,626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-01,638430,1055,21995.756,1023,895.0,95,286,3.5,56.5,260558,96.957495,2.9350,0.10,15553.9,2598.4,1754.4,1036
2020-09-01,657613,1121,23179.135,1105,933.0,96,286,2.5,56.6,260742,97.144271,2.8900,0.09,15643.4,2506.2,1774.6,991
2020-10-01,677725,1141,23846.574,1162,898.0,100,284,3.5,57.4,260925,97.335974,2.8340,0.09,15568.4,2370.9,1794.0,1001


### Quarterly datasets

In [86]:
quarter_df = [vacancy_df,

            week_earning,

            delinquent_rate,

            hor_qtr]

In [87]:
qtr_2000 = vacancy_df.join(quarter_df[1:], how='inner')

In [88]:
qtr_2000

,owner_vacancy,rent_vacancy,week_earning,delinquent_rate,hor
date,,,,,
2000-01-01,1.6,7.9,603,1.95,67.100000
2000-04-01,1.5,8.0,606,1.97,67.200000
2000-07-01,1.6,8.2,611,2.09,67.700000
2000-10-01,1.6,7.8,614,2.23,67.500000
2001-01-01,1.5,8.2,620,2.34,67.500000
...,...,...,...,...,...
2020-10-01,1.0,6.5,1029,2.73,65.822503
2021-01-01,0.9,6.8,1040,2.68,65.600000
2021-04-01,0.9,6.2,1048,2.48,65.400000


# Save Datasets locally

> Do not run if you don't need the datasets to be saved

In [95]:
def save_data(d:pd.DataFrame, name:str, path:str):
    file = handle_file(path, name)
    d.to_csv(file, index_label='date')

In [102]:
save_data(monthly_1990, 'monthly_1990.csv', './data')

Warning !
A file exists with the same name as the one being saved
Should I override the existing file (y/n): y


In [103]:
save_data(qtr_2000, 'quarterly_2000.csv', './data')

Warning !
A file exists with the same name as the one being saved
Should I override the existing file (y/n): y


In [104]:
save_data(hp_idx, 'hpi_monthly.csv', './data')

Warning !
A file exists with the same name as the one being saved
Should I override the existing file (y/n): y


In [105]:
save_data(hp_idx_qtr, 'hpi_quarterly.csv', './data')

Warning !
A file exists with the same name as the one being saved
Should I override the existing file (y/n): y
